In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, roc_curve, accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer, LabelBinarizer
import category_encoders

import time 
import pickle

import matplotlib.pyplot as plt 
%matplotlib inline 

Functions

In [41]:
def cross_val(X, y, model, kf):
    X, y = np.array(X), np.array(y).reshape(-1)
    cv_scores = np.zeros((5,4), dtype=np.float32)

    for i, (train_index, val_index) in enumerate(kf.split(X, y)):

        print( "Fold ", i)

        y_train, y_val = y[train_index].copy(), y[val_index].copy()
        X_train, X_val = X[train_index, :].copy(), X[val_index, :].copy()
        
        fit_model = model.fit(X_train, y_train)
        pred = fit_model.predict(X_val)

        cv_scores[i, :] = [f1_score(y_val, pred), 
                           precision_score(y_val, pred), 
                           recall_score(y_val, pred), 
                           roc_auc_score(y_val, pred)]
        
    return cv_scores

def print_metrics(cv_scores):
    metrics = ['f1', 'precision', 'recall', 'roc auc']
    cvmean = cv_scores.mean(0)
    for i in range(4):
        print("{} = {:.5f}".format(metrics[i], cvmean[i]))


Load data

In [60]:
test = pd.read_csv('./input/orange_small_churn_test_data.csv')
train, target = pd.read_csv('./input/orange_small_churn_data.train'), \
np.where(pd.read_csv('./input/orange_small_churn_labels.train', header=-1)==1, 1, 0).ravel()

test_id= test['ID']
test.drop(['ID'], axis=1, inplace=True)

Preprocessing

In [61]:
# # check duplicates 
# # rows
# np.sum(train.duplicated()) 
# # columns
# arr = np.array(train)
# dup = []
# for i in range(arr.shape[1]):
#     for j in range(i+1, arr.shape[1]):
#         if np.all(np.equal(arr[:, i], arr[:, j])):
#             dup.append((i, j))
# dup # [(197, 219), (197, 221), (199, 213), (219, 221)]

In [79]:
def transform(train, test, target, to_drop=False, 
              high_cardinality="smoothing", hc_treshold = 10, hc_drop=False, # high cardinality categorical
              eb_k=50, eb_f=10,  # parameters for hc smoothing function 
              encode=False,  # categorical 
              fill_num=-1, scaling=False  # continuous 
             ):
    
    """ 
    data preprocessing 
    
    :train, test: pandas DataFrame
    :high_cardinality: way to handle categorical features with high number of levels
    :encode: category encoding, 'ohe' = one hot, 'bin' = binary
    :fill_num: fill nan for continuous features, -1 = with -1, ('mean', 'median') = strategy
    :scaling: 'standard' = StandartScaler
    
    category features should have type 'object'
    """

    # remove duplicates 
    if to_drop:
        train = train.drop(to_drop, axis=1)
        test = test.drop(to_drop, axis=1)
    
    ######## categorical features 
    
    cat_features = train.columns[train.dtypes=='object']
    num_features = train.columns[train.dtypes!='object']      
        
    # factorize 
    le = LabelEncoder()
    train[cat_features] = train[cat_features].fillna('-1')
    test[cat_features] = test[cat_features].fillna('-1')
    for c in cat_features:
        data=train[c].append(test[c])
        le.fit(data.values.tolist())  # nan = 0 level
        train[c] = le.transform(train[c].values.tolist())
        test[c] = le.transform(test[c].values.tolist())       
    
    # mark nan with -1, if encoding not necessary 
    if not encode:
        train[cat_features] = train[cat_features].replace(0, -1)
        test[cat_features] = test[cat_features].replace(0, -1)        
    ######## high cardinality
    
    if high_cardinality:

        hc_features = train[cat_features].columns[train[cat_features].apply(lambda x: len(x.value_counts())) > hc_treshold]
        target_mean = target.mean()
        S = {}

        for c in hc_features:

            if high_cardinality == "sr":
                # supervised ratio 
                group_means = pd.concat([train[c], pd.DataFrame(target, columns=['target'], index=train.index)], axis=1).groupby(c).mean()
                group_means = group_means.target.to_dict()
                for group in train[c].value_counts().index:
                    S[group] = group_means[group]

            if high_cardinality=="woe":
                # weight of evidence
                group_y1 = pd.concat([train[c], pd.DataFrame(target, columns=['target'], index=train.index)], axis=1).\
                groupby([c]).agg('sum')
                group_y0 = pd.concat([train[c], pd.DataFrame(target, columns=['target'], index=train.index)], axis=1).\
                groupby([c]).agg('count') - group_y1
                y1 = (target==1).sum()
                y0 = (target==0).sum()
                woe = np.log(((group_y1) / y1) / ((group_y0) / y0))
                for i,v in zip(woe.index, np.where(np.isinf(woe), 0, woe)):
                    S[i] = v[0]

            if high_cardinality=="smoothing":
                # empirical bayes (smoothing for small group)
                group_means = pd.concat([train[c], pd.DataFrame(target, columns=['target'], index=train.index)], axis=1).groupby(c).mean()
                group_means = group_means.target.to_dict()
                group_counts = pd.concat([train[c], pd.DataFrame(target, columns=['target'], index=train.index)], axis=1).groupby(c).agg('count')
                group_counts = group_counts.target.to_dict()

                def smoothing_function(n, k, f):
                    return 1 / (1 + np.exp(-(n-k)/f))

                for group in train[c].value_counts().index:
                    lam = smoothing_function(n=group_counts[group], k=eb_k, f=eb_f)
                    S[group] = lam*group_means[group] + (1-lam)*target_mean

            # transform train
            train[c+'_avg'] = train[c].apply(lambda x: S[x]).copy()

            # transform test
            def hc_transform_test(x):
                if x in S: 
                    return S[x]
                else:
                    return target_mean

            test[c+'_avg'] = test[c].apply(hc_transform_test).copy()

        # drop hc features 
        if hc_drop:
            train.drop(hc_features, axis=1, inplace=True)
            test.drop(hc_features, axis=1, inplace=True)

        # update cat features 
        cat_features = sorted(list(set(cat_features).difference(hc_features)))

    ######## for linear models 
    
    # fill missings
    if fill_num in ['mean', 'median']:
        imputer = Imputer(strategy=fill_num)
        train[num_features] = imputer.fit_transform(train[num_features])
        test[num_features] = imputer.transform(test[num_features])
    elif fill_num < 0:
        train[num_features] = train[num_features].fillna(fill_num)
        test[num_features] = test[num_features].fillna(fill_num)
        
    # scaling
    if scaling=='standard':
        scaler = StandardScaler()
        train[num_features] = scaler.fit_transform(train[num_features])
        test[num_features] = scaler.transform(test[num_features])
    
    ######## encoding 
    if encode=='ohe':
        # one hot encoding, memory inefficient
        oh = OneHotEncoder(sparse=False)
        for c in cat_features:
            data=train[c].append(test[c])
            oh.fit(data.reshape(-1,1))            
            train_temp = oh.transform(train[c].reshape(-1,1))
            test_temp = oh.transform(test[c].reshape(-1,1))
            train = pd.concat([train, pd.DataFrame(train_temp, 
                                                   columns=[(c+"_"+str(i)) for i in data.value_counts().index],
                                                   index = train.index
                                                  )], axis=1)
            test = pd.concat([test, pd.DataFrame(test_temp, 
                                                 columns=[(c+"_"+str(i)) for i in data.value_counts().index],
                                                 index = test.index
                                                )], axis=1)
            # drop column
            train.drop(c, axis=1, inplace=True)
            test.drop(c, axis=1, inplace=True)
    
    if encode=='bin':
        # binary encoding 
        pass
            
    return train, test

In [80]:
# train, test = transform(train, test, target, 
#                         encode='ohe', scaling=True, fill_num='median', hc_drop=True,
#                         to_drop=['Var214', 'Var220', 'Var222'])
# train.shape, test.shape 

In [81]:
# X.isnull().sum().value_counts()[:5]
# nan_group_1 = X[X.columns[X.isnull().sum() == 34506]].dropna().index
# nan_group_2 = X[X.columns[X.isnull().sum() == 34141]].dropna().index
# nan_group_3 = X[X.columns[X.isnull().sum() == 33975]].dropna().index
# nan_group_4 = X[X.columns[X.isnull().sum() == 33881]].dropna().index
# set(nan_group_3).intersection(list(nan_group_2))
# выставим порог отсева пустых значений на 99%
# X = X[X.columns[(X.isnull().sum() / X.shape[0] ) < 0.75]].copy()

Cross-validation

In [90]:
MISS_THERSHOLD = 0.9
ENCODE = "ohe"
FILL_NUM = 'median'
HC = "woe"
HC_DROP = True
HC_K = 50
HC_F = 10

############################################################################################################
# load 
test = pd.read_csv('./input/orange_small_churn_test_data.csv')
train, target = pd.read_csv('./input/orange_small_churn_data.train'), \
np.where(pd.read_csv('./input/orange_small_churn_labels.train', header=-1)==1, 1, 0).ravel()

test_id= test['ID']
test.drop(['ID'], axis=1, inplace=True)

# выставим порог отсева пустых значений на 95%
to_drop = train.columns[(train.isnull().sum() / train.shape[0] ) >= MISS_THERSHOLD]
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)

# transform
train, test = transform(train, test, target, 
                        encode=ENCODE, scaling=True, fill_num=FILL_NUM, 
                        hc_drop=HC_DROP, high_cardinality=HC, eb_k=HC_K, eb_f=HC_F,
                        to_drop=['Var214', 'Var220', 'Var222'])
print(train.shape, test.shape )

# split data 
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=5000, random_state=42, stratify=target)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2017)
model = LogisticRegression(class_weight='balanced')

# cv 
cv_scores = cross_val(X_train, y_train, model, kf)
print_metrics(cv_scores)

C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in log
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:128: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:129: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:130: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


(40000, 122) (10000, 122)
Fold  0
Fold  1
Fold  2
Fold  3
Fold  4
f1 = 0.19412
precision = 0.11515
recall = 0.61828
roc auc = 0.61804


In [85]:
# model.fit(X_train, y_train)

In [86]:
# predict_proba = model.predict_proba(X_valid)[:, 1]
# fpr, tpr, _ = roc_curve(y_valid, predict_proba)
# roc_auc_score(y_valid, predict_proba)

In [87]:
# plt.plot(fpr, tpr)
# plt.plot([0,1],[0,1],'r--')

In [88]:
# cutoff = np.linspace(0.45, 0.6, 15)
# for cut in cutoff:
#     print("f1: {:.3}\trecall: {:.3}\tprecision: {:.3}\tauc: {:.3}\tacc: {:.3}\tcutoff: {:.2} ".format(
#         f1_score(y_valid, predict_proba > cut), 
#         recall_score(y_valid, predict_proba > cut), 
#         precision_score(y_valid, predict_proba > cut), 
#         roc_auc_score(y_valid, predict_proba > cut),
#         accuracy_score(y_valid, predict_proba>cut),
#         cut
#     ))

In [97]:
predict = model.predict(test)

In [100]:
# save submission
sub = pd.concat([test_id, pd.DataFrame(np.where(predict==0, -1, 1), columns=['result'])], axis=1)
sub.to_csv('./input/sub_lr.csv', index=False)